# table of contents
1. [predictions, part III](#predictions,-part-III)
2. [preprocessing](#preprocessing)
3. [oversampling](#oversampling)
4. [undersampling](#undersampling)
5. [SMOTE](#SMOTE)

# Predictions, part III
- drop columns: no
- scaling: yes
- hyperparameter tuning: yes
- one-hot encoding: yes, the dataset was found encoded
- `resampling: yes`

In this session, I'm addressing class imbalance in the target category.\
Target column "is_canceled" has 37% vs 63% for canceled vs not canceled bookings.\
I'm using 3 techniques:
1. oversampling,
2. undersampling,
3. SMOTE.

`The main takeaway from this session is that resampling doesn't improve performance of my models.`

# preprocessing

In [ ]:
# import libraries
%run common_imports.py

# load and split data
%run load_and_split_data.py
X_train, X_test, y_train, y_test = load_and_split_data()

# scale data
%run minmaxscaler.py
X_train_scaled, X_test_scaled = scale_data(X_train, X_test)

# oversampling

In [ ]:
# Make a copy of X_train to avoid altering the original DataFrame
X_train_cl = X_train.copy()

# Add the 'is_canceled' column to X_train_cl
X_train_cl["is_canceled"] = y_train.values

# Separate canceled and not_canceled instances
canceled = X_train_cl[X_train_cl["is_canceled"] == 1]
not_canceled = X_train_cl[X_train_cl["is_canceled"] == 0]

# Visualize class distribution
canceled_plt = X_train_cl["is_canceled"].value_counts()
canceled_plt.plot(kind="bar")
plt.title("Class Distribution Before Oversampling")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

# Oversample the minority class
canceled_oversampled = resample(canceled,
                                replace=True,
                                n_samples=len(not_canceled),
                                random_state=0)

# Concatenate oversampled minority class with majority class
train_over = pd.concat([canceled_oversampled, not_canceled])

# Update X_train and y_train with the oversampled data
X_train = train_over.drop(columns=["is_canceled"])
y_train = train_over["is_canceled"]

# Standardize the oversampled data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verify the shapes of X_train and y_train
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

# Visualize class distribution after oversampling
canceled_plt = train_over["is_canceled"].value_counts()
canceled_plt.plot(kind="bar")
plt.title("Class Distribution After Oversampling")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

In [ ]:
# Call the function to process the hyperparameter tuning results
from train_with_best_hyperparameters import process_hyperparameter_tuning_results

process_hyperparameter_tuning_results(
    input_file="../data/hyperparameter_tuning_results.csv", 
    output_file="../data/accuracies_with_parameters_oversampled.csv",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    X_train_scaled=X_train_scaled,
    X_test_scaled=X_test_scaled,
    y_test=y_test
)

# undersampling

In [ ]:
# Make a copy of X_train to avoid altering the original DataFrame
X_train_copy = X_train.copy()

# Add the 'is_canceled' column to X_train_copy
X_train_copy["is_canceled"] = y_train.values

# Separate canceled and not_canceled instances
canceled = X_train_copy[X_train_copy["is_canceled"] == 1]
not_canceled = X_train_copy[X_train_copy["is_canceled"] == 0]

# Visualize class distribution
canceled_plt = X_train_copy["is_canceled"].value_counts()
canceled_plt.plot(kind="bar")
plt.title("Class Distribution Before Undersampling")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

# Undersample the majority class
not_canceled_undersampled = resample(not_canceled,
                                     replace=False,
                                     n_samples=len(canceled),
                                     random_state=0)

# Concatenate undersampled majority class with minority class
train_under = pd.concat([not_canceled_undersampled, canceled])

# Update X_train and y_train with the undersampled data
X_train = train_under.drop(columns=["is_canceled"])
y_train = train_under["is_canceled"]

# Standardize the undersampled data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verify the shapes of X_train and y_train
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

# Visualize class distribution after undersampling
canceled_plt = train_under["is_canceled"].value_counts()
canceled_plt.plot(kind="bar")
plt.title("Class Distribution After Undersampling")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

In [ ]:
# Call the function to process the hyperparameter tuning results
from train_with_best_hyperparameters import process_hyperparameter_tuning_results

process_hyperparameter_tuning_results(
    input_file="../data/hyperparameter_tuning_results.csv", 
    output_file="../data/accuracies_with_parameters_undersampled.csv",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    X_train_scaled=X_train_scaled,
    X_test_scaled=X_test_scaled,
    y_test=y_test
)

# SMOTE

In [ ]:
# Visualize class distribution before SMOTE
canceled_plt_before = y_train.value_counts()
canceled_plt_before.plot(kind="bar")
plt.title("Class Distribution Before SMOTE")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

# Apply SMOTE and reassign to the original variable names
sm = SMOTE(random_state=1, sampling_strategy=1.0)
X_train, y_train = sm.fit_resample(X_train, y_train)

# Standardize the resampled data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Assuming X_test needs to be standardized similarly

# Visualize class distribution after SMOTE
canceled_plt_after = y_train.value_counts()
canceled_plt_after.plot(kind="bar")
plt.title("Class Distribution After SMOTE")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

In [ ]:
# Call the function to process the hyperparameter tuning results
from train_with_best_hyperparameters import process_hyperparameter_tuning_results

process_hyperparameter_tuning_results(
    input_file="../data/hyperparameter_tuning_results.csv", 
    output_file="../data/accuracies_with_parameters_smote.csv",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    X_train_scaled=X_train_scaled,
    X_test_scaled=X_test_scaled,
    y_test=y_test
)

In [ ]:
# Load the data
accuracies_with_parameters = pd.read_csv("../data/accuracies_with_parameters.csv")
accuracies_with_parameters_oversampled = pd.read_csv("../data/accuracies_with_parameters_oversampled.csv")
accuracies_with_parameters_undersampled = pd.read_csv("../data/accuracies_with_parameters_undersampled.csv")
accuracies_with_parameters_smote = pd.read_csv("../data/accuracies_with_parameters_smote.csv")

# Identify the best accuracy for each model in each dataframe
idx = accuracies_with_parameters.groupby("model")["accuracy_in_%"].idxmax()
best_params = accuracies_with_parameters.loc[idx, ["model", "best_parameters", "accuracy_in_%", "source"]].reset_index(drop=True)
best_params["dataset"] = "original"

idx_oversampled = accuracies_with_parameters_oversampled.groupby("model")["accuracy_in_%"].idxmax()
best_params_oversampled = accuracies_with_parameters_oversampled.loc[idx_oversampled, ["model", "best_parameters", "accuracy_in_%", "source"]].reset_index(drop=True)
best_params_oversampled["dataset"] = "oversampled"

idx_undersampled = accuracies_with_parameters_undersampled.groupby("model")["accuracy_in_%"].idxmax()
best_params_undersampled = accuracies_with_parameters_undersampled.loc[idx_undersampled, ["model", "best_parameters", "accuracy_in_%", "source"]].reset_index(drop=True)
best_params_undersampled["dataset"] = "undersampled"

idx_smote = accuracies_with_parameters_smote.groupby("model")["accuracy_in_%"].idxmax()
best_params_smote = accuracies_with_parameters_smote.loc[idx_smote, ["model", "best_parameters", "accuracy_in_%", "source"]].reset_index(drop=True)
best_params_smote["dataset"] = "smote"

# Combine the results into a single dataframe
combined_results = pd.concat([best_params, best_params_oversampled, best_params_undersampled, best_params_smote], ignore_index=True)

# Sort the combined results
accuracies_sorted = combined_results.sort_values(by=["model", "accuracy_in_%", "dataset"], ascending=[True, False, True])

# Save the sorted dataframe
accuracies_sorted.to_csv("../data/accuracies_resampled.csv", index=False)

# Display the dataframe
accuracies_sorted